In [ ]:
import pandas as pd
import numpy as np
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
import visualizer_helpers as vh

pd.set_option('display.max_rows',100)

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

In [ ]:
base_trip = pd.read_csv(f"{base_dir}/final_trips.csv", 
                        index_col='trip_id', 
                        usecols=['trip_id', 'trip_mode', 'trip_count','trip_num', 'trip_count', 'origin','destination', 'purpose','depart'])
base_trip_idx = base_trip.index
build_trip = pd.read_csv(f"{build_dir}/final_trips.csv", 
                         index_col='trip_id', 
                         usecols=['trip_id', 'trip_mode', 'trip_count','trip_num', 'trip_count', 'origin','destination', 'purpose','depart'])


In [ ]:
print(f"""Filtering all trips by origin {
    'and' if how_method == 'all' else 'or' if how_method == 'any' else '???'
    } destination MAZ.\nOriginal trips in base: {len(base_trip)}\tbuild: {len(build_trip)}\tdiff: {len(build_trip)-len(base_trip)}""")

base_trip = base_trip[multi_filter_mazs([base_trip.origin, base_trip.destination])] # base tour in the filtered set

build_trip = build_trip[

    (build_trip.index.isin(base_trip_idx) # originally existed in base
    & build_trip.index.isin(base_trip.index) )    # and was in the current set for the base
 
    |     # OR
    ((~build_trip.index.isin(base_trip_idx)) # is a new tour 
     & multi_filter_mazs([build_trip.origin, build_trip.destination])) # and it's in this set
     
    ]

print(f"After filtering, total trips in base: {len(base_trip)}\tbuild: {len(build_trip)}\tdiff: {len(build_trip)-len(base_trip)}")

## Trip Purpose

In [ ]:
# To analyze "intermediate" trip purposes, count only:
# Trips on tours multiple stops (trip_count > 1) and 
# Excluding the last stop on the tour (trip_count != trip_num)
df = pd.merge(
    base_trip[(base_trip.trip_count > 1) & (base_trip.trip_count != base_trip.trip_num)],
    build_trip[(build_trip.trip_count > 1) & (build_trip.trip_count != build_trip.trip_num)],
    how='outer',
    left_index=True,
    right_index=True,
    suffixes=('_base', '_build')
)

In [ ]:
df.loc[df.purpose_base == df.purpose_build,'Case'] = "Unchanged"
df.loc[df.purpose_base != df.purpose_build,'Case'] = "Changed"

df.loc[df.purpose_base.isna(),'Case'] = "Newly Created"
df.loc[df.purpose_build.isna(),"Case"] = "Removed"

In [ ]:
df_case = df.Case.value_counts()
df_case

In [ ]:
fig = vh.create_pie_chart(df_case.to_frame().sort_index(),["count"])
fig.show()

## Trip Destination

In [ ]:
df = base_trip[base_trip.trip_count != base_trip.trip_num].merge(build_trip[build_trip.trip_count != build_trip.trip_num],
                                                                 how='outer',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))


In [ ]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [ ]:
df = df.Case.value_counts()

df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## Trip Scheduling

In [ ]:
# translate departure to time period index
base_trip['depart_period'] = base_trip.depart.apply(vh.get_time_period_index)
build_trip['depart_period'] = build_trip.depart.apply(vh.get_time_period_index)

# join base and build trips on trip_id
# compare trips that are in both base and build
df = base_trip.merge(
    build_trip, 
    how='inner', 
    left_index=True, 
    right_index=True,
    suffixes=('_base', '_build')
)

# get the difference in departure time
df['depart_bin_difference'] = df.depart_build - df.depart_base
# get the difference in departure time period
df['depart_period_difference'] = df.depart_period_build - df.depart_period_base

In [ ]:
mapper = {
    0: "EA",
    1: "AM",
    2: "MD",
    3: "PM",
    4: "EV"
}
tp_order = ['EA','AM','MD','PM','EV','Total']

metric='depart'
purpose_df = df
display(Markdown(f"### Trip departure changes"))
xtab = pd.crosstab(
    purpose_df[f'{metric}_period_base'].replace(mapper),
    purpose_df[f'{metric}_period_build'].replace(mapper),
    margins=True,
    margins_name='Total'
)
display(xtab.loc[tp_order,tp_order])

summary_text = f"""**Trip departure changes:**
    Trips which departed earlier in build: {len(purpose_df[purpose_df[f'{metric}_bin_difference'] < 0])}
    Trips which departed in the same bin in build: {len(purpose_df[purpose_df[f'{metric}_bin_difference'] == 0])}
    Trips which departed later in build: {len(purpose_df[purpose_df[f'{metric}_bin_difference'] > 0])}

"""
display(Markdown(summary_text.replace("\n","<br>")))

## Trip Mode Choice

In [ ]:
df = base_trip[['trip_mode', 'purpose']].merge(build_trip[['trip_mode', 'purpose']],
                                                                 how='outer',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))

df.loc[df.trip_mode_base == df.trip_mode_build,'Case'] = "Unchanged"
df.loc[df.trip_mode_base != df.trip_mode_build,'Case'] = "Changed"

df.loc[df.trip_mode_base.isna(),'Case'] = "Newly Created"
df.loc[df.trip_mode_build.isna(),"Case"] = "Removed"

In [ ]:
fig = vh.create_pie_chart(df_case.to_frame(), ["count"])
fig.show()

In [ ]:
combiners = {
    "EBIKE": "MICROMOBILITY",
    "ESCOOTER": "MICROMOBILITY",

    "WALK_LOC": "WALK_TRANSIT",
    "WALK_PRM": "WALK_TRANSIT",
    "WALK_MIX": "WALK_TRANSIT",

    "PNR_LOC": "PNR_TRANSIT",
    "PNR_PRM": "PNR_TRANSIT",
    "PNR_MIX": "PNR_TRANSIT",

    "KNR_LOC": "KNR_TRANSIT",
    "KNR_PRM": "KNR_TRANSIT",
    "KNR_MIX": "KNR_TRANSIT",

    "TNC_LOC": "TNC_TRANSIT",
    "TNC_PRM": "TNC_TRANSIT",
    "TNC_MIX": "TNC_TRANSIT",

    "TAXI": "RIDESHARE",
    "TNC_SHARED": "RIDESHARE",
    "TNC_SINGLE": "RIDESHARE",

}
order = ['DRIVEALONE','SHARED2','SHARED3',
         'WALK','BIKE','MICROMOBILITY','WALK_TRANSIT',
         'PNR_TRANSIT','KNR_TRANSIT','TNC_TRANSIT',
         'RIDESHARE','SCH_BUS','Newly created','Removed',"Total",]


In [ ]:
transit_modes = [f'{access_mode}_{transit_mode}' 
                 for access_mode in ['WALK','PNR','KNR','TNC'] 
                 for transit_mode in ['LOC','PRM','MIX']]
auto_modes = ['DRIVEALONE','SHARED2','SHARED3']
nmot_modes = ['BIKE','EBIKE','ESCOOTER','WALK']
rideshare_modes = ['TAXI','TNC_SHARED','TNC_SINGLE']

In [ ]:
df[df.trip_mode_build.isin(transit_modes)
   &(~(df.trip_mode_base.isin(transit_modes)))

   ].trip_mode_base.value_counts()

In [ ]:
summary_text = f"""**Trip mode changes:**
To transit
<p style="margin-left: 40px">
from auto: {len(df[df.trip_mode_base.isin(auto_modes) & df.trip_mode_build.isin(transit_modes)])}
from non-motorized: {len(df[df.trip_mode_base.isin(nmot_modes) & df.trip_mode_build.isin(transit_modes)])}
from rideshare: {len(df[df.trip_mode_base.isin(rideshare_modes) & df.trip_mode_build.isin(transit_modes)])}
from school bus: {len(df[df.trip_mode_base.isin(['SCH_BUS']) & df.trip_mode_build.isin(transit_modes)])}
from new tour: {len(df[df.trip_mode_base.isna() & df.trip_mode_build.isin(transit_modes)])}
*Subtotal: {len(df[df.trip_mode_build.isin(transit_modes)&(~(df.trip_mode_base.isin(transit_modes)))])}*
</p>
Stayed on transit: {len(df[df.trip_mode_base.isin(transit_modes) & df.trip_mode_build.isin(transit_modes)])}
Stayed on something else: {len(df[~(df.trip_mode_base.isin(transit_modes)|df.trip_mode_build.isin(transit_modes))])}
Switched to something else: {len(df[df.trip_mode_base.isin(transit_modes)&(~(df.trip_mode_build.isin(transit_modes)))])}
"""
display(Markdown(summary_text.replace("\n","<br>")))

In [ ]:
# xtab = df[['trip_mode_base', 'trip_mode_build']].replace(combiners).value_counts().unstack().fillna(0)
# xtab.loc[order,order]

order = pd.Series(order)
xtab = pd.crosstab(
    df.trip_mode_base.replace(combiners).fillna('Newly created'),
      df.trip_mode_build.replace(combiners).fillna('Removed'),
      margins=True,
      margins_name='Total')

display(xtab.loc[order[order.isin(xtab.index)],order[order.isin(xtab.columns)]])

In [ ]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

# for now, loop through the the recorded trip purposes
# note that keying against purpose_base will only include trips that exist in base 
all_purps = df.purpose_base.sort_values().unique()

for purp in all_purps:
    df_purp = df.loc[df.purpose_base == purp]
    if len(df_purp) == 0:
        continue 
        
    print(f"#### {purp}")
    df_purp_cases = df_purp.Case.value_counts()
    print(str(df_purp_cases).replace("\n","<br>"))
    
    fig = vh.create_pie_chart(df_purp_cases.to_frame().sort_index(),["count"])
    
    fig.show()
    display(Markdown(" "))